In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
# Install tmdbsimple (only need to run once)
!pip install tqdm

In [14]:
# Load API Credentials
with open(r'/Users/mmenj/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['API Key'], timeout_s=5.0)

In [15]:
# set our API call parameters 
LOCATION = 'SAN ANTONIO,TX'
TERM = 'Taco'

In [16]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_NY_pizza.json"
JSON_FILE

'Data/results_in_progress_SA_taco.json'

In [19]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [20]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [21]:
## How many results total?
total_results = results['total']
total_results

3100

In [22]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [23]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

155

In [24]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [25]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [26]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/155 [00:00<?, ?it/s]

In [70]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)

    ## If it DOES exist:
    if file_exists == True:

        ## Check if user wants to delete if exists
        if delete_if_exists==True:

            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            


    ## If it does NOT exist:
    else:

        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")

        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)

        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  



In [71]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[i] Data/results_in_progress_SA_taco.json not found. Saving empty list to new file.
- 0 previous results found.


155

In [72]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/155 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [73]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,fD626-6wEZgfGP2YT3wSIA,taquitos-west-ave-san-antonio,Taquitos West Ave,https://s3-media2.fl.yelpcdn.com/bphoto/yWNgVE...,False,https://www.yelp.com/biz/taquitos-west-ave-san...,726,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.5,"{'latitude': 29.48637, 'longitude': -98.52573}",[delivery],$,"{'address1': '2818 West Ave', 'address2': None...",+12105259888,(210) 525-9888,1118.116301
1,zulD0Ns_524wMuCAFlXxRA,tlahco-mexican-kitchen-san-antonio,Tlahco Mexican kitchen,https://s3-media4.fl.yelpcdn.com/bphoto/PAEto8...,False,https://www.yelp.com/biz/tlahco-mexican-kitche...,725,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.5,"{'latitude': 29.5006724654481, 'longitude': -9...","[pickup, delivery]",$$,"{'address1': '6702 San Pedro Ave', 'address2':...",+12104550135,(210) 455-0135,2931.033269
2,j2Ym1bd-tRyM-G5qXZ1yCw,petes-tako-house-san-antonio,Pete's Tako House,https://s3-media1.fl.yelpcdn.com/bphoto/EzV5R6...,False,https://www.yelp.com/biz/petes-tako-house-san-...,1288,"[{'alias': 'breakfast_brunch', 'title': 'Break...",4.5,"{'latitude': 29.4328575134277, 'longitude': -9...","[pickup, delivery]",$,"{'address1': '502 Brooklyn Ave', 'address2': '...",+12102242911,(210) 224-2911,6094.469626
3,62rv8KQaduDNpTAOxSD1NA,tacos-and-chelas-mexican-kitchen-san-antonio,Tacos and Chelas Mexican Kitchen,https://s3-media1.fl.yelpcdn.com/bphoto/nDbKGh...,False,https://www.yelp.com/biz/tacos-and-chelas-mexi...,410,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.0,"{'latitude': 29.435665, 'longitude': -98.686426}","[pickup, delivery]",$$,"{'address1': '9902 Potranco Rd', 'address2': N...",+12103849145,(210) 384-9145,17101.082307
4,vsmZf-TpPtYhJP-OByDmKg,san-taco-san-antonio,San taco,https://s3-media1.fl.yelpcdn.com/bphoto/rxUZMz...,False,https://www.yelp.com/biz/san-taco-san-antonio?...,22,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.5,"{'latitude': 29.44262, 'longitude': -98.504275}",[],NaN,"{'address1': '114 Fredericksburg Rd', 'address...",+12103143099,(210) 314-3099,4294.718174


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,lq704t40dIE7Zf-3gEsNlQ,tacon-madre-new-braunfels,Tacon madre,https://s3-media4.fl.yelpcdn.com/bphoto/UFjVtW...,False,https://www.yelp.com/biz/tacon-madre-new-braun...,2,"[{'alias': 'foodtrucks', 'title': 'Food Trucks...",4.5,"{'latitude': 29.7025, 'longitude': -98.09702}",[],NaN,"{'address1': '201 Loop 337', 'address2': None,...",+17012609947,(701) 260-9947,47586.868015
996,DFD2goCw5yyeiJy8jKlwAA,miguels-country-kitchen-hondo,Miguel's Country Kitchen,https://s3-media3.fl.yelpcdn.com/bphoto/HZx9Aq...,False,https://www.yelp.com/biz/miguels-country-kitch...,9,"[{'alias': 'mexican', 'title': 'Mexican'}]",2.5,"{'latitude': 29.347575, 'longitude': -99.1294}",[],$,"{'address1': '209 US Highway 90 E', 'address2'...",+18307418641,(830) 741-8641,61093.799478
997,j_wNJeXW6y3Zk7Tl2hv6zw,la-joya-mexican-restaurant-san-antonio,La Joya Mexican Restaurant,https://s3-media2.fl.yelpcdn.com/bphoto/MF-Sjt...,False,https://www.yelp.com/biz/la-joya-mexican-resta...,4,"[{'alias': 'mexican', 'title': 'Mexican'}]",5.0,"{'latitude': 29.4246254, 'longitude': -98.5547...",[],NaN,"{'address1': '510 S General McMullen Dr', 'add...",+12104365188,(210) 436-5188,7103.042383
998,VUMbigFrt_NwQpaedFZQMw,karolinas-mexican-food-san-antonio,Karolina's Mexican Food,https://s3-media2.fl.yelpcdn.com/bphoto/KP99Z4...,False,https://www.yelp.com/biz/karolinas-mexican-foo...,13,"[{'alias': 'mexican', 'title': 'Mexican'}]",3.5,"{'latitude': 29.3032457440659, 'longitude': -9...",[delivery],NaN,"{'address1': '11510 US 281', 'address2': '', '...",+12109735910,(210) 973-5910,20087.463958
999,Wa7d0oUPh7pRd8PO9qx1vg,santi-s-san-marcos,Santi’s,https://s3-media2.fl.yelpcdn.com/bphoto/7-CABt...,False,https://www.yelp.com/biz/santi-s-san-marcos?ad...,1,"[{'alias': 'tacos', 'title': 'Tacos'}, {'alias...",5.0,"{'latitude': 29.88169, 'longitude': -97.94149}",[],NaN,"{'address1': '126 S Guadalupe St', 'address2':...",,,71349.575881


In [76]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()

0

In [77]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_SA_Taco.csv.gz', compression='gzip',index=False)